# Tutorial Prefect + Dagshub + mlflow + DVC

## Integración y Funcionamiento de Herramientas en el Pipeline

* **Prefect:** Orquesta el pipeline, organizando el flujo de tareas.
    * **@task:** Cada función que realiza una operación específica se decora con @task lo que permite que Prefect gestione la ejecución, el manejo de errores y la monitorización de cada tarea.
    * **@Flow** Se utiliza para definir el flujo principal, que organiza la ejecución de todas las tareas en el orden deseado.
    
* **Dagshub:** Facilita la colaboración, el versionado de datos y la integración con MLflow.
    * para **inicializar dagshub** usamos la función dagshub.init(...)  que se invoca al inicio del flujo para conectar el proyecto con el repositorio y habilitar la integración con MLflow.

* **MLflow:** Registra y rastrea experimentos, parámetros, métricas y modelos.
    * **Registro de Experimentos:**
        * Se **inicia mlflow** con mlflow.start_run(...) para cada modelo, lo que permite agrupar todos los registros asociados a ese experimento.
    
    * **Registro de Parámetros y Métricas:**
        * Los parámetros del modelo se registran mediante mlflow.log_params(...).
        * Las métricas de evaluación (MSE, RMSE, MAE, R2) se registran utilizando mlflow.log_metrics(...).
    
    * **Almacenamiento de Modelos:**
        * Una vez entrenado, el modelo se registra y almacena usando mlflow.sklearn.log_model(...), lo que facilita su posterior carga y despliegue.

## Apartados del notebook:
1. Cargado y procesado de datos.
2. dividir los datos en conjuntos de entrenamiento y prueba.
3. Entrenar y evaluar distintos modelos de regresion.
4. Registrar los experimentos y modelos utilizando MLFlow y DagsHub.

Cada uno de estos pasos se implementa como una tarea (`@task`) de Prefect y se orquesta mediante un flujo principal (`@flow`).

## Importar Librerias

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

import mlflow
import mlflow.sklearn
import dagshub

from prefect import flow, task

## Cargar y preprocesar los datos


In [ ]:
@task
def load_and_preprocess_data(filepath: str) -> pd.DataFrame:
    data = pd.read_csv(filepath, delimiter=';', decimal=',')
    
    data = data[data['Hora'].between(0, 23)]
    
    data['DateTime'] = pd.to_datetime(
        data['Fecha'] + ' ' + data['Hora'].astype(str) + ':00:00', 
        format='%d/%m/%Y %H:%M:%S'
    )
    
    data = data.set_index('DateTime')
    data = data.sort_index()
    
    return data

## Dividir los datos en entrenamiento y prueba


In [ ]:
@task
def split_data(data: pd.DataFrame):
    y = data['Consumo_kWh']
    
    X = data.index.astype(np.int64).values.reshape(-1, 1)
    
    train_size = int(len(X) * 0.7)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    
    return X_train, X_test, y_train, y_test

## Entrenar y registrar un modelo con MLflow
1. **Inicio de una Corrida de Experimentación:**
   * **`with mlflow.start_run(run_name=model_name) as run:`**  
     Este bloque inicia una nueva corrida de experimentación en MLflow. Al asignar el nombre del modelo, se facilita la identificación de los experimentos en la interfaz de MLflow.

2. **Registro de Metadatos y Parámetros:**
   * **`mlflow.set_tag("model_name", model_name)`**  
     Se añade una etiqueta (tag) con el nombre del modelo, lo cual ayuda a clasificar y buscar experimentos específicos.
   * **`mlflow.log_params(params)`**  
     Con esta línea se registran todos los parámetros del modelo. Esto permite documentar las configuraciones con las que se entrenó el modelo.

3. **Entrenamiento del Modelo:**
   * Aunque el entrenamiento (con model.fit(X_train, y_train)) se realiza de forma habitual, el hecho de estar dentro del contexto de MLflow garantiza que el proceso quede registrado, asociando el entrenamiento con los parámetros y la versión del experimento.

4. **Evaluación y Registro de Métricas:**
   * Después de realizar las predicciones y calcular las métricas de evaluación (MSE, RMSE, MAE, R2), se registra cada una de estas métricas:

5. **Almacenamiento y Versionado del Modelo:**
   * **`mlflow.sklearn.log_model(model, "model", input_example=X_train)`**  
     Esta línea registra y almacena el modelo entrenado en MLflow. Utilizando el método específico para modelos de scikit-learn, se guarda el modelo junto con un ejemplo de entrada.

6. **Retorno de las Métricas:**
   * la función devuelbe un diccionario con las métricas calculadas.


In [ ]:
@task
def train_and_log_model(model_name: str, params: dict, model, X_train, y_train, X_test, y_test):
    with mlflow.start_run(run_name=model_name) as run:
        mlflow.set_tag("model_name", model_name)
        mlflow.log_params(params)

        model.set_params(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        mlflow.log_metrics({
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2
        })

        mlflow.sklearn.log_model(model, "model", input_example=X_train)
    
    return {"MSE": mse, "RMSE": rmse, "MAE": mae, "R2": r2}

## Flujo Principal

* Inicializa DagsHub y configura el experimento en MLflow.
* Llama a la tarea de carga y preprocesamiento.
* Divide los datos en conjuntos de entrenamiento y prueba.
* Define una lista de modelos a entrenar, junto con sus parámetros.
* Para cada modelo, ejecuta la tarea `train_and_log_model` y almacena las métricas resultantes.


In [ ]:
@flow(name="Flujo de entrenamiento y evaluación de modelos")
def main_flow(filepath: str):
    dagshub.init(repo_owner='auditoria.SGBA1', repo_name='SGBA1-smartgrids', mlflow=True)
    mlflow.set_experiment("Prueba 3 - Mlflow, DagsHub y Factura_Luz")
    
    data = load_and_preprocess_data(filepath)
    
    X_train, X_test, y_train, y_test = split_data(data)
    
    models = [
        ("Linear Regression", {}, LinearRegression()),
        ("Random Forest Regressor", {"n_estimators": 30, "max_depth": 3}, RandomForestRegressor()),
        ("XGBRegressor", {"use_label_encoder": False, "eval_metric": 'rmse'}, XGBRegressor())
    ]
    
    results = {}
    
    for model_name, params, model in models:
        metrics = train_and_log_model(model_name, params, model, X_train, y_train, X_test, y_test)
        results[model_name] = metrics
    
    return results

In [ ]:
if __name__ == "__main__":
    filepath = 'data/varillas_24-25.csv'

    results = main_flow(filepath)
    print("Resultados de la evaluación de modelos:")
    for model, metrics in results.items():
        print(f"{model}: {metrics}")

21:04:41.867 | INFO    | Flow run 'daring-jackal' - Beginning flow run 'daring-jackal' for flow 'Flujo de entrenamiento y evaluación de modelos'

Initialized MLflow to track repo "auditoria.SGBA1/SGBA1-smartgrids"

Repository auditoria.SGBA1/SGBA1-smartgrids initialized!

21:04:43.025 | INFO    | Task run 'load_and_preprocess_data-837' - Finished in state Completed()

21:04:43.369 | INFO    | Task run 'split_data-7cc' - Finished in state Completed()

...

21:05:04.348 | INFO    | Task run 'train_and_log_model-2fa' - Finished in state Completed()

🏃 View run XGBRegressor at: https://dagshub.com/auditoria.SGBA1/SGBA1-smartgrids.mlflow/#/experiments/4/runs/9b5920f4544f4b0d8526409cc6508697
🧪 View experiment at: https://dagshub.com/auditoria.SGBA1/SGBA1-smartgrids.mlflow/#/experiments/4


21:05:18.981 | INFO    | Task run 'train_and_log_model-56a' - Finished in state Completed()

21:05:19.056 | INFO    | Flow run 'daring-jackal' - Finished in state Completed()

Resultados de la evaluación de modelos:
Linear Regression: {'MSE': 0.05142135341179214, 'RMSE': np.float64(0.22676276901597436), 'MAE': 0.1213649807816132, 'R2': -0.1450123137302124}
Random Forest Regressor: {'MSE': 0.1964072471551676, 'RMSE': np.float64(0.4431785725361365), 'MAE': 0.4219112909764975, 'R2': -3.373449969267978}
XGBRegressor: {'MSE': 0.04902147673288318, 'RMSE': np.float64(0.2214079418920721), 'MAE': 0.16596375114410422, 'R2': -0.09157365125901884}
